# Semi-automatization of DRN and SFR sensitivity analysis/calibration

**Worflow**\
DRN
1. Load .drn file - *completed*
2. Change the conductance of the cells - *completed*
3. Write the .drn file - *completed*
4. Run the model - *in progress*
5. Read .cbb file
6. Store the outflow data of the first X cells
7. Put the process in a loop over a range of parameters

SFR
1. Load .sfr file - *in progress*
2. Change the hydraulic conductivity of the cells
3. Write the .sfr file
4. Run the model
5. Read streamflow.dat file
6. Store the flux data of cell at reach X
7. Put the process in a loop over a range of parameters

## Setup

In [1]:
import flopy
import flopy.utils.binaryfile as bf
import glob
import numpy as np
import os
import pandas as pd

#check if the below lines are still useful at the end
import sys
from pprint import pformat
from tempfile import TemporaryDirectory

In [2]:
# cwd = "C:/Users/user/OneDrive - Politecnico di Milano/hydrogeo-modelling/TEAM_Idrogeo/Tesi/Tesi_Ceola_Pirovano/Tesi_Pirovano/busca_drain_start"
cwd = os.getcwd()

In [3]:
def import_input_file(path):
    f = open(path).readlines()
    df = pd.DataFrame()
    for row in f[4:]:
        r = list(filter(None, row.split(' ')))
        df = pd.concat([df, pd.DataFrame(r).transpose()])
    df.columns = ['layer', 'row', 'column', 'stage', 'conductance', 'node']
    df.node = df.node.str.removesuffix('\n')
    df.layer = df.layer.astype('int')
    df.row = df.row.astype('int')
    df.column = df.column.astype('int')
    df.stage = df.stage.astype('float')
    df.conductance = df.conductance.astype('float')
    df.reset_index(inplace=True, drop = True)
    return df

## DRAIN

### Load .drn file

In [34]:
# Load .drn file
drn = import_input_file(os.path.join(cwd, 'test_files', 'busca_drain.drn'))
drn.head()

,layer,row,column,stage,conductance,node
0,1,46,51,124.838,0.012,0
1,1,47,51,124.835,0.018,0
2,1,48,51,124.832,0.006,0
3,1,48,52,124.830,0.024,0
4,1,49,52,124.824,0.018,0


In [5]:
# Load DRAIN cells characteristics
drn_sp = pd.read_csv(os.path.join(cwd, 'test_files', 'busca_drain_specifiche_celle.csv'))

In [6]:
# Compute the conductance of each cell for a given hydraulic conductivity
# Change the conductance column
k = 0.0001 #hydraulic conductivity
conductance_change = (k*drn_sp.Width*drn_sp.Length)/drn_sp.Thickness
drn.conductance = conductance_change

### Write .drn file using flopy

**bug found:** the layer gets put equal to 2 instead of 1
- the problem is not in the stress_period_data table passed to drn
- check stress_period_data.write_transient method
- couldn't figure it out, so found a way out: set drn.layer = 0
- drn_input_file_test_v4.drn works

In [42]:
modelpth = os.path.join(cwd, 'test_files')

# create the model class (only useful because it's needed by ModflowDrn class)
# this can be replaced by loading the actual model .nam file (not needed though)
mf = flopy.modflow.Modflow(
     "drn_test",
    model_ws = modelpth,
    exe_name = "mf2005",
)

c:\Users\user\anaconda3\envs\fontanili\Lib\site-packages\flopy\mbase.py:97: UserWarning: The program mf2005 does not exist or is not executable.
  warn(


In [43]:
# transform the drn structure in the flopy required format for "stress_period_data"
drn.layer = 0
stress_period_data = {0: drn.iloc[:, :-1].to_numpy().tolist()} #remove node column, not needed nor supported by flopy's ModflowDrn class

In [50]:
#generate the drn package inside the flopy class
ipakcb = 50 #code for cell-by-cell flow data storage
drain = flopy.modflow.ModflowDrn(mf, ipakcb=ipakcb, stress_period_data=stress_period_data,
                                 filenames=os.path.join(cwd, 'test_files', 'drn_input_file_test_v5.drn'))
drain.write_file(check = False)

### Test: Run the model

In [6]:
# trial: run from flopy

# load the existing model busca_drain
# https://flopy.readthedocs.io/en/3.3.2/source/flopy.modflow.mf.html

mf = flopy.modflow.Modflow.load(
        os.path.join(cwd, 'test_files', 'drain_model_test', 'busca_drain.nam'),
        model_ws = os.path.join(cwd, 'test_files', 'model_test'),
        exe_name='MF2005',
        version = 'mf2005',
        verbose = False
        )

In [7]:
# drn stress period data can be accessed by:
mf.drn.stress_period_data.data

{0: rec.array([(0,  45,  50, 124.838, 0.012, 0.),
            (0,  46,  50, 124.835, 0.018, 0.),
            (0,  47,  50, 124.832, 0.006, 0.),
            (0,  47,  51, 124.83 , 0.024, 0.),
            (0,  48,  51, 124.824, 0.018, 0.),
            (0,  48,  52, 124.822, 0.006, 0.),
            (0,  49,  52, 124.818, 0.024, 0.),
            (0,  50,  52, 124.814, 0.012, 0.),
            (0,  50,  53, 124.811, 0.006, 0.),
            (0,  51,  53, 124.807, 0.024, 0.),
            (0,  52,  53, 124.803, 0.006, 0.),
            (0,  52,  54, 124.8  , 0.012, 0.),
            (0,  53,  54, 124.796, 0.018, 0.),
            (0,  54,  54, 124.792, 0.006, 0.),
            (0,  54,  55, 124.79 , 0.018, 0.),
            (0,  55,  55, 124.785, 0.018, 0.),
            (0,  56,  55, 124.782, 0.006, 0.),
            (0,  56,  56, 124.779, 0.024, 0.),
            (0,  57,  56, 124.774, 0.018, 0.),
            (0,  57,  57, 124.771, 0.006, 0.),
            (0,  58,  57, 124.768, 0.024, 0.),
          

In [ ]:
# this doen't work, doesn't find the executable
success, buff = mf.run_model()
if not success:
    raise Exception("MODFLOW did not terminate normally.")

In [5]:
# this works
success, buff = flopy.mbase.run_model(
                exe_name = os.path.join(cwd, 'test_files', 'drain_model_test','MF2005.exe'),
                namefile = 'busca_drain.nam',
                model_ws = os.path.join(cwd, 'test_files', 'drain_model_test')
                )
if not success:
    raise Exception("MODFLOW did not terminate normally.")

FloPy is using the following executable to run the model: MF2005.exe

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.11.00 8/8/2013                        

 Using NAME file: busca_drain.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2024/10/29 14:22:11

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2024/10/29 14:22:12
 Elapsed run time:  0.269 Seconds

  Normal termination of simulation


In [24]:
# run by launching mf2005.exe directly
os.startfile(os.path.join(cwd, 'test_files', 'drain_model_test','MF2005.exe'))
# the .exe is opened, but then error messages arise for missing .dll files

### Read .cbb output file

class CellBudgetFile\
https://flopy.readthedocs.io/en/latest/source/flopy.utils.binaryfile.html

The outflow will have to be considered until cell:
- row 92, column 76 (last cell of first segment)
Check if any substantial change in outflow happen if considering:
- row 93, column 76 (first cell of second segment)

In [19]:
cbb = bf.CellBudgetFile(os.path.join(cwd, 'test_files', 'drain_model_test', 'busca_drain.cbb'))

In [22]:
cbb.headers

,kstp,kper,text,ncol,nrow,nlay
36,1,1,CONSTANT HEAD,196,321,1
251736,1,1,FLOW RIGHT FACE,196,321,1
503436,1,1,FLOW FRONT FACE,196,321,1
755136,1,1,DRAINS,196,321,1
1006836,1,1,RECHARGE,196,321,1


In [24]:
drain = cbb.get_data(text = 'DRAINS')

In [29]:
drain[0].shape

(1, 321, 196)

In [32]:
drain[0][0, 92, 76]

np.float32(-0.0015603425)

In [35]:
# get all the cells until 92, 76
# sum the flux extracted from drain[0]
drn

,layer,row,column,stage,conductance,node
0,1,46,51,124.838,0.012,0
1,1,47,51,124.835,0.018,0
2,1,48,51,124.832,0.006,0
3,1,48,52,124.830,0.024,0
4,1,49,52,124.824,0.018,0
...,...,...,...,...,...,...
323,1,264,156,124.041,0.012,0
324,1,265,156,124.035,0.018,0
325,1,266,156,124.027,0.018,0
326,1,266,157,124.023,0.006,0


## SFR

### Test: load .sfr file

In [ ]:
#the file is multi structured:
# - the above section has a structure similar to .drn
# - in the last section, the structure is different: there is the info about the segments
# - width and manning coefficient are defined in the segment section
# - length, slope, bed thickness and bed hydraulic conductivity are defined in the first section
# try to use flopy to load the model and then modify the sfr parameters directly from there

# another possibility:
# - export sfr as .txt from the model
# - read its characteristics here
# - create a tool flopy model
# - add sfr to the tool model
# - generate the input file

In [9]:
sfr = open(os.path.join(cwd, 'test_files', 'busca_sfr2.sfr')).readlines()
sfr

['# SFR Package for MODFLOW2000\n',
 ' -328  7  0  0  1.000000e+00  1.000000e-04  50  54 1 10 1 30\n',
 '  1  46  51  1  1  2.000000  124.837925  0.001590  0.500000  0.001000\n',
 '  1  47  51  1  2  3.000000  124.835267  0.001590  0.500000  0.001000\n',
 '  1  48  51  1  3  1.000000  124.832472  0.001590  0.500000  0.001000\n',
 '  1  48  52  1  4  4.000000  124.829516  0.001590  0.500000  0.001000\n',
 '  1  49  52  1  5  3.000000  124.824446  0.001590  0.500000  0.001000\n',
 '  1  49  53  1  6  1.000000  124.821639  0.001590  0.500000  0.001000\n',
 '  1  50  53  1  7  4.000000  124.818274  0.001590  0.500000  0.001000\n',
 '  1  51  53  1  8  2.000000  124.813762  0.001590  0.500000  0.001000\n',
 '  1  51  54  1  9  1.000000  124.810955  0.001590  0.500000  0.001000\n',
 '  1  52  54  1  10  4.000000  124.807045  0.001590  0.500000  0.001000\n',
 '  1  53  54  1  11  1.000000  124.803078  0.001590  0.500000  0.001000\n',
 '  1  53  55  1  12  2.000000  124.800271  0.001590  0.500

In [10]:
#error when loading the sfr package
mf = flopy.modflow.Modflow.load(
        f = os.path.join(cwd, 'test_files', 'sfr_model_test', 'busca_sfr2.nam'),
        # model_ws = os.path.join(cwd, 'test_files', 'sfr_model_test'),
        # exe_name='MF2005',
        # version = 'mf2005',
        verbose = True
        )


Creating new model with name: busca_sfr2
--------------------------------------------------

Parsing the namefile --> c:\repos\hm\Fontanili\test_files\sfr_model_test\busca_sfr2.nam

--------------------------------------------------
External unit dictionary:
{7: filename:c:\repos\hm\Fontanili\test_files\sfr_model_test\busca_sfr2.lst, filetype:LIST, 1: filename:c:\repos\hm\Fontanili\test_files\sfr_model_test\busca_sfr2.bas, filetype:BAS6, 29: filename:c:\repos\hm\Fontanili\test_files\sfr_model_test\busca_sfr2.dis, filetype:DIS, 11: filename:c:\repos\hm\Fontanili\test_files\sfr_model_test\busca_sfr2.lpf, filetype:LPF, 27: filename:c:\repos\hm\Fontanili\test_files\sfr_model_test\busca_sfr2.sfr, filetype:SFR, 18: filename:c:\repos\hm\Fontanili\test_files\sfr_model_test\busca_sfr2.rch, filetype:RCH, 22: filename:c:\repos\hm\Fontanili\test_files\sfr_model_test\busca_sfr2.oc, filetype:OC, 19: filename:c:\repos\hm\Fontanili\test_files\sfr_model_test\busca_sfr2.pcg, filetype:PCG, 50: filename:

IndexError: pop from empty list

In [16]:
#this works now, because we changed the input file structure to SFR1 from GWV
#next step:
# - identify the parameters inside the sfr class
# - modify the k
# - save the new .sfr file
m = flopy.modflow.Modflow()
f = os.path.join(cwd, 'test_files', 'sfr_model_test', 'test_sfr1_input', 'busca_sfr2_tp.sfr')

sfr = flopy.modflow.ModflowSfr2.load(f, m, gwt =True)

In [17]:
sfr


Streamflow-Routing (SFR2) Package Class

Parameters
----------
model : model object
    The model object (of type :class:'flopy.modflow.mf.Modflow') to which
    this package will be added.
nstrm : integer
    An integer value that can be specified to be positive or negative. The
    absolute value of NSTRM is equal to the number of stream reaches
    (finite-difference cells) that are active during the simulation and
    the number of lines of data to be included in Item 2, described below.
    When NSTRM is specified to be a negative integer, it is also used as a
    flag for changing the format of the data input, for simulating
    unsaturated flow beneath streams, and (or) for simulating transient
    streamflow routing (for MODFLOW-2005 simulations only), depending
    on the values specified for variables ISFROPT and IRTFLG, as described
    below. When NSTRM is negative, NSFRPAR must be set to zero, which means
    that parameters cannot be specified. By default, nstrm is set t

In [18]:
sfr.reach_data

rec.array([(0, 0,  45,  50, 1,  1, 2., 0., 0., 0., 0., 0., 0., 0., 0.,   1,   2),
           (0, 0,  46,  50, 1,  2, 3., 0., 0., 0., 0., 0., 0., 0., 0.,   2,   3),
           (0, 0,  47,  50, 1,  3, 1., 0., 0., 0., 0., 0., 0., 0., 0.,   3,   4),
           (0, 0,  47,  51, 1,  4, 4., 0., 0., 0., 0., 0., 0., 0., 0.,   4,   5),
           (0, 0,  48,  51, 1,  5, 3., 0., 0., 0., 0., 0., 0., 0., 0.,   5,   6),
           (0, 0,  48,  52, 1,  6, 1., 0., 0., 0., 0., 0., 0., 0., 0.,   6,   7),
           (0, 0,  49,  52, 1,  7, 4., 0., 0., 0., 0., 0., 0., 0., 0.,   7,   8),
           (0, 0,  50,  52, 1,  8, 2., 0., 0., 0., 0., 0., 0., 0., 0.,   8,   9),
           (0, 0,  50,  53, 1,  9, 1., 0., 0., 0., 0., 0., 0., 0., 0.,   9,  10),
           (0, 0,  51,  53, 1, 10, 4., 0., 0., 0., 0., 0., 0., 0., 0.,  10,  11),
           (0, 0,  52,  53, 1, 11, 1., 0., 0., 0., 0., 0., 0., 0., 0.,  11,  12),
           (0, 0,  52,  54, 1, 12, 2., 0., 0., 0., 0., 0., 0., 0., 0.,  12,  13),
           (0, 0

### Store the flow at reach x

Consider the outward flow of these two cells:
- reach 72, segment 1
- reach 1, segment 2

## Other tests

### Test: write .drn file autonomously without using flopy

Riprovare con il pacchetto di flopy usando un numpy array invecedel dataframe


Da qui stavo scrivendo write_file per scrivermi i dati
https://flopy.readthedocs.io/en/latest/_modules/flopy/modflow/mfdrn.html#ModflowDrn.write_file

stavo indagando come flopy gestisce lo stress_period_data (MFils o qualcosa)

In [ ]:
ipakcb = 1
line = f"{2:10d}{ipakcb:10d}"
line

'         2         1'

In [ ]:
heading = '# DRN package for MODFLOW-2005 generated by paolchol'
n_drain_cells = drn.shape[0]
ipakcb = 50


with open(os.path.join(cwd, 'drn_input_file_test.drn'), 'w') as file:
    file.write(f"{heading}\n")
    line = f"{n_drain_cells:10d}{ipakcb:10d}"
    # for opt in self.options:
        # line += " " + str(opt)
    line += "\n"
    file.write(line)

    # for row in drn.iterrows():
    #     file.writelines(row)

,index,layer,row,column,stage,conductance,node
0,0,1,46,51,124.838,0.0012,0
1,0,1,47,51,124.835,0.0018,0
2,0,1,48,51,124.832,0.0006,0
3,0,1,48,52,124.830,0.0024,0
4,0,1,49,52,124.824,0.0018,0
...,...,...,...,...,...,...,...
323,0,1,264,156,124.041,0.0012,0
324,0,1,265,156,124.035,0.0018,0
325,0,1,266,156,124.027,0.0018,0
326,0,1,266,157,124.023,0.0006,0
